In [221]:
from scipy.linalg import cho_solve, cho_factor
import numpy as np
from tqdm import tqdm

def FitStump(X, Y, w, gamma):
    # fist derive b
    prediction = X @ w
    sorting = np.argsort(prediction)
    b = - prediction[sorting]
    b = np.append(b[0]+ np.sign(b[0]) * 1, b)
    gamma_sorted = gamma[sorting]
    Y_sorted = Y[sorting]

    # c_upper = - np.cumsum(gamma_sorted * Y_sorted)
    c_upper =  np.append(0, np.cumsum(gamma_sorted * Y_sorted)) - np.sum(gamma_sorted * Y_sorted)
    # c_lower = np.reverse(np.cumsum(gamma))
    c_lower = np.sum(gamma) - np.append(0, np.cumsum(gamma))
    # print(c_lower.shape)
    c = c_upper / np.maximum(c_lower, 1e-5)
    a = np.sum(gamma * (Y + c.T), axis=0) / np.maximum(np.append(0, np.cumsum(gamma)), 1e-5)
    # print("Computed c and a", c.shape, a.shape, b.shape)
    # print((np.add.outer(X @ w , b.T) > 0).shape)
    # # print(((np.add.outer(X @ w , b.T) > 0) * a.T).shape)
    # print((Y - (np.add.outer(X @ w , b.T) > 0) * a.T + c.T).shape)

    errors = np.linalg.norm(gamma * (Y - (np.add.outer(X @ w , b.T) >= 0) * a.T + c.T), axis=0)
    print(errors)

    # for i, (a_value,b_value,c_value) in enumerate(zip(a, b, c)):
    #     error = np.linalg.norm(gamma * (Y - (np.add.outer(X @ w , b.T) > 0) * a.T + c.T), axis=0)
    #     print(i)
    # errors = np.array([np.linalg.norm(gamma * (Y - a_value * ((X @ w + b_value) > 0) + c_value)) for a_value,b_value,c_value in zip(a, b, c)])

    min_index = np.argmin(errors)
    return a[min_index], b[min_index], c[min_index], errors[min_index]



    # for b in range(b_values):
    #     # now optimize 
    #     c = np.sum(gamma * Y * (((X @ w + b) > 0) - 1))/np.max(np.sum(gamma * (1 - ((X @ w + b) > 0))), 1e-10)
    #     a = np.sum(gamma * (Y + c))/np.max(np.sum(gamma * ((X @ w + b) > 0)), 1e-10)
    #     min_error = np.linalg.norm(gamma * (Y - a * ((X @ w + b) > 0) + c))
    # return a, b, c, min_error


def GentleBoost(X, Y, k):
    dim = X.shape[1]
    gamma = np.ones((X.shape[0], 1))/X.shape[0]
    a_s = []
    b_s = []
    c_s = []
    w_s = []
    predictions = np.zeros_like(Y, dtype=float)
    for episode in tqdm(range(k)):
        w = np.random.randn(dim)
        w = w/np.linalg.norm(w)
        a,b,c,error = FitStump(X, Y, w, gamma)
        w_s.append(w)
        a_s.append(a)
        b_s.append(b)
        c_s.append(c)
        current_prediction =  a *(((X @ w) + b) > 0) + c
        gamma = gamma * np.expand_dims(np.exp(- np.squeeze(Y) * current_prediction), axis=1)
        # print(np.expand_dims(np.exp(- np.squeeze(Y) * current_prediction), axis=1))
        gamma = gamma/np.sum(gamma)
        # print(np.sum(gamma))
        # print(current_prediction.shape)
        predictions += np.expand_dims(current_prediction, axis=1)
        # print(current_prediction)
        print(f"The training error is:{np.linalg.norm(gamma * (Y - predictions))}")



    return w_s, a_s, b_s, c_s

In [209]:
data = np.load("USPS_data.npy", allow_pickle=True).item()
X_train = data["Xtrain"]
Y_train = data["Ytrain"]
X_train = X_train[np.where(Y_train < 2)[0], :]
Y_train = Y_train[np.where(Y_train < 2)[0]]
X_test = data["Xtest"]
Y_test = data["Ytest"]

In [222]:
w, a, b, c = GentleBoost(X_train, Y_train, 20)

 15%|█▌        | 3/20 [00:00<00:00, 25.07it/s]

[0.01865761 0.02524185 0.02524994 0.02525807 0.02526624 0.02527446
 0.02528272 0.02529102 0.02529937 0.02530776 0.0253162  0.02532468
 0.0253332  0.02534178 0.0253504  0.02535906 0.02536777 0.02537653
 0.02538533 0.02539419 0.02540309 0.02541203 0.02542103 0.02543007
 0.02543917 0.02544831 0.0254575  0.02546674 0.02547603 0.02548537
 0.02549476 0.0255042  0.02551369 0.02552323 0.02553283 0.02554247
 0.02555217 0.02556192 0.02557173 0.02558159 0.0255915  0.02560146
 0.02561148 0.02562155 0.02563168 0.02564186 0.0256521  0.02566239
 0.02567274 0.02568315 0.02569361 0.02570413 0.02571471 0.02572534
 0.02573604 0.02574679 0.0257576  0.02576847 0.0257794  0.02579039
 0.02580144 0.02581255 0.02582372 0.02583495 0.02584624 0.0258576
 0.02586902 0.0258805  0.02589204 0.02590365 0.02591532 0.02592706
 0.02593886 0.02595073 0.02596266 0.02597466 0.02598672 0.02599886
 0.02601105 0.02602332 0.02603565 0.02604806 0.02606053 0.02607307
 0.02608568 0.02609836 0.02611112 0.02612394 0.02613683 0.02614

 45%|████▌     | 9/20 [00:00<00:00, 25.25it/s]

The training error is:0.023001053494391323
[3.59930641e-52 3.59930641e-52 2.66329347e-52 2.66614295e-52
 2.66903355e-52 2.67189161e-52 2.67474661e-52 2.67768730e-52
 2.68055091e-52 2.68341146e-52 2.68626896e-52 2.68912343e-52
 2.69213793e-52 2.69518006e-52 2.69825009e-52 2.70134830e-52
 2.70447495e-52 2.70763033e-52 2.71055239e-52 2.71374876e-52
 2.71697449e-52 2.71991739e-52 2.72285711e-52 2.72613663e-52
 2.72944621e-52 2.73278615e-52 2.73615675e-52 2.73955830e-52
 2.74299111e-52 2.74645550e-52 2.74995177e-52 2.75298644e-52
 2.75652746e-52 2.76010106e-52 2.76315752e-52 2.76621061e-52
 2.76926032e-52 2.77290476e-52 2.77658263e-52 2.78029426e-52
 2.78337860e-52 2.78713713e-52 2.79023083e-52 2.79403670e-52
 2.79787749e-52 2.80099345e-52 2.80488245e-52 2.80800788e-52
 2.81194556e-52 2.81508048e-52 2.81906731e-52 2.82309068e-52
 2.82715097e-52 2.83032164e-52 2.83348876e-52 2.83761273e-52
 2.84177450e-52 2.84496449e-52 2.84917773e-52 2.85342960e-52
 2.85772049e-52 2.86094697e-52 2.86416982e

 75%|███████▌  | 15/20 [00:00<00:00, 25.48it/s]

The training error is:0.023001053494391323
[3.59930641e-52 7.52967886e-50 6.41675935e-52 6.41675935e-52
 5.12084999e-52 6.41108703e-52 7.76093550e-52 6.41107847e-52
 7.30636262e-52 8.21908406e-52 8.21908406e-52 9.13402614e-52
 7.75302095e-52 6.40535818e-52 5.96250307e-52 5.65500452e-52
 5.42744189e-52 5.25250737e-52 5.53325360e-52 5.36421238e-52
 5.22718839e-52 5.45617846e-52 5.45617846e-52 5.31835782e-52
 5.20605984e-52 5.11048496e-52 5.02819062e-52 4.95661269e-52
 4.89380326e-52 4.83825678e-52 4.78879177e-52 4.78879177e-52
 4.74174488e-52 4.70188561e-52 4.81610416e-52 4.93154678e-52
 5.04812947e-52 4.99496978e-52 4.94662886e-52 4.90248565e-52
 5.00418781e-52 4.95930175e-52 5.05737391e-52 5.01198255e-52
 4.97009759e-52 5.06095558e-52 5.01865781e-52 5.01865781e-52
 4.97612680e-52 5.06058532e-52 4.93966939e-52 4.90251977e-52
 4.87078930e-52 4.94649490e-52 5.02268722e-52 4.98794999e-52
 4.95534406e-52 5.02709134e-52 4.99419470e-52 4.96320446e-52
 4.93396109e-52 4.99975617e-52 5.06590398e

100%|██████████| 20/20 [00:00<00:00, 27.43it/s]

The training error is:0.023001053494391323
[3.59930641e-52 3.59930641e-52 2.66329347e-52 2.66614295e-52
 3.03942026e-52 3.94735974e-52 5.11414317e-52 4.70745495e-52
 5.53338280e-52 6.39970157e-52 6.39970157e-52 7.28485265e-52
 6.39397113e-52 5.87120196e-52 5.52913902e-52 4.93500404e-52
 4.80212076e-52 4.70220253e-52 4.97064895e-52 4.62298644e-52
 4.55638549e-52 4.77195588e-52 4.99211396e-52 4.69952658e-52
 4.63617739e-52 4.58441896e-52 4.38565195e-52 4.35518741e-52
 4.19808673e-52 4.18207309e-52 4.05423159e-52 4.05304933e-52
 4.04618308e-52 4.04104368e-52 4.14417490e-52 4.24922418e-52
 4.35605276e-52 4.33645737e-52 4.22156266e-52 4.20791318e-52
 4.30048858e-52 4.28553811e-52 4.37544677e-52 4.35801335e-52
 4.25903655e-52 4.25731879e-52 4.24384965e-52 4.32321456e-52
 4.30940312e-52 4.38677443e-52 4.37116839e-52 4.35659762e-52
 4.34296252e-52 4.41337354e-52 4.41337354e-52 4.39622842e-52
 4.38211004e-52 4.44857870e-52 4.43326034e-52 4.41881911e-52
 4.40518188e-52 4.46642211e-52 4.52815678e